## Laboratorium 5 - algorytm Najbliższej Średniej (NM)


### Opis
Celem laboratorium jest implementacja klasyfikatora najbliższej średniej NM (*Nearest Mean*).


### Zadanie 1
* Wczytaj dane.
* Wszystkie poniższe zadania wykonaj dla wszystkich dostępnych klas i wszystkich cech.


In [1]:
import numpy as np
with open('./dataset/dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

### Zadanie 2
Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Euklidesa i wykonaj klasyfikację. Wyświetl wynik klasyfikacji (accuracy).

In [2]:
#Adjusting data
#####################################################
import sklearn as skl

y_train = train[:,0].copy()
x_train = train[:,2:].copy()

y_test = test[:,0].copy()
x_test = test[:,2:].copy()

print("Liczba klas: :", len(np.unique(y_train)))

print("Dlugosc zbiory treningowego (labels):",len(y_train))
print("Dlugosc zbiory treningowego (data):",len(x_train))

print("Dlugosc zbiory testowego (labels):",len(y_test))
print("Dlugosc zbiory testowego (data):",len(x_test))



    


Liczba klas: : 30
Dlugosc zbiory treningowego (labels): 2244
Dlugosc zbiory treningowego (data): 2244
Dlugosc zbiory testowego (labels): 1496
Dlugosc zbiory testowego (data): 1496


In [3]:
"""
#Normalize data 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#Fit on train data
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

print("Wartosc min w zbiorze treningowym: ",np.min(x_train))
print("Wartosc max w zbiorze treningowym: ",np.max(x_train))

print("Wartosc min w zbiorze testowym: ",np.min(x_test))
print("Wartosc max w zbiorze testowym: ",np.max(x_test))
"""

'\n#Normalize data 0-1\nfrom sklearn.preprocessing import MinMaxScaler\nscaler = MinMaxScaler()\n#Fit on train data\nscaler.fit(x_train)\n\nx_train = scaler.transform(x_train)\nx_test = scaler.transform(x_test)\n\nprint("Wartosc min w zbiorze treningowym: ",np.min(x_train))\nprint("Wartosc max w zbiorze treningowym: ",np.max(x_train))\n\nprint("Wartosc min w zbiorze testowym: ",np.min(x_test))\nprint("Wartosc max w zbiorze testowym: ",np.max(x_test))\n'

In [4]:
#Definicja NM
##############################################################################

#Stworzenie centroidow
classes = np.unique(y_train)
centroids = []

for c in classes:
    binary_vector = y_train == c
    class_centroid = x_train[binary_vector].mean(axis = 0)
    centroids.append(class_centroid)


#Funkcja nearest_mean
def Nearest_Mean(centroids, sample, classes):
    distances_to_class = [] 
    for centroid in centroids:
        #Obliczenie dystansu
        distance = float(np.linalg.norm(sample - centroid))
        distances_to_class.append(distance)
        
    #Znalezienie najmniejszego dystansu
    idx = int(np.argmin(distances_to_class))
    chosen_class = classes[idx]
        
    return chosen_class

Predykcja

In [5]:
#Predykcja
y_pred = []
for sample in x_test:
    pred = Nearest_Mean(centroids,sample, classes)
    y_pred.append(pred)

#To array    
y_pred = np.array(y_pred)
    

acc = skl.metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy of test set prediction is: {acc:.4f}")

Accuracy of test set prediction is: 0.4606


### Zadanie 3
Zaimplementuj funkcję, która zwraca macierz kowariancji (*uwaga: biblioteka `numpy` posiada gotową implementację `cov` z którą powinieneś porównać swój wynik*).

\begin{equation*}
C = \frac{1}{n - 1} (X - \bar X)(X - \bar X)^T
\end{equation*}

gdzie:
* $X$ to macierz danych,
* $\bar X$ to wektor średnich wartości cech. 



In [6]:
#Funkcja na macierz kowariancji
def create_cov_matrix(matrix):
    #X - X_mean
    matrix_centered = matrix - matrix.mean(axis=0)
    
    cov = (matrix_centered.T @ matrix_centered) / (matrix.shape[0] - 1)
    return cov
    

#Macierz kowariancji wersja numpy vs wlasna
np_cov_matrix = np.cov(x_train, rowvar=False)
own_cov_matrix = create_cov_matrix(x_train)

equal = np.allclose(np_cov_matrix, own_cov_matrix)

print(f"Covariance matrices are equal: {equal}")

Covariance matrices are equal: True


### Zadanie 4
Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Mahalanobisa i wykonaj klasyfikację. Wyświetl wynik klasyfikacji (accuracy).

\begin{equation*}
D_j = \sqrt{ (x - \mu_j)^T S_j^{-1}(x - \mu_j) },
\end{equation*}

gdzie:
* $D_j$ to odległość klasyfikowanej próbki od klasy $j$, 
* $\mu_j$ to wektor średnich wartości cech dla klasy $j$, 
* $S_j^{-1}$ to macierz odwrotna do macierzy kowariancji klasy $j$, 
* a $x$ to klasyfikowana próbka.

> Podpowiedź: Do obliczenia macierzy odwrotnej możesz użyć funkcji `linalg.inv` z biblioteki `numpy`.

> UWAGA: W niniejszym zadaniu możesz zastosować dowolną strukturę kodu (nie musisz trzymać się struktury z poprzedniego zadania), jednak algorytm NM należy zaimplementować samodzielnie – bez użycia gotowych rozwiązań (np. z biblioteki `scikit-learn`).

<span style="text-decoration:underline">Referencje</span>

1. Mahalanobis, P C, _On test and measures of group divergence : theoretical formulae_, Journal and Proceedings of Asiatic Society of Bengal (New Series) Vol. 26, pp. 541-588. 1930. (URL: http://library.isical.ac.in:8080/xmlui/bitstream/handle/10263/1639/029.pdf)
2. McLachlan, Goeffrey J. _Mahalanobis distance_, Resonance, pp. 20-26. 1999. (URL: https://www.ias.ac.in/article/fulltext/reso/004/06/0020-0026)

In [7]:
########Mahalonobis
#Macierze kowariancji dla każdej klasy
cov_matrices = []
for c in classes:
    binary_vector = y_train == c
    cov = create_cov_matrix(x_train[binary_vector])
    cov_matrices.append(cov)

#Funkcja
def Nearest_Mean_Mah(centroids,cov_matrices, sample, classes):
    distances_to_class = [] 
    #Calculate distances
    for centroid,cov_matrix in zip(centroids,cov_matrices):
        d = sample - centroid
        
        mah_distance = d.T @ np.linalg.inv(cov_matrix) @ d
        mah_distance = np.sqrt(mah_distance)
        distances_to_class.append(float(mah_distance))
        
    #Znalezienie najmniejszego dystansu
    idx = int(np.argmin(distances_to_class))
    chosen_class = classes[idx]
    return chosen_class




In [8]:
#Predykcja
y_pred_mah = []
for sample in x_test:
    pred = Nearest_Mean_Mah(centroids,cov_matrices, sample, classes)
    y_pred_mah.append(pred)

#To array    
y_pred_mah = np.array(y_pred_mah)

acc_mah = skl.metrics.accuracy_score(y_test, y_pred_mah)
print(f"Accuracy of test set prediction is: {acc_mah:.4f}")

Accuracy of test set prediction is: 0.5635
